In [ ]:
import pandas as pd
import numpy as np

# ----------------------------
# LOAD & PREP
# ----------------------------
df = pd.read_csv("Walmart_Store_sales.csv")
df['Date'] = pd.to_datetime(df['Date'], format='%d-%m-%Y')
df = df.sort_values(['Store','Date']).reset_index(drop=True)

# Sanity: expected columns
expected = {'Store','Date','Weekly_Sales','Holiday_Flag','Temperature','Fuel_Price','CPI','Unemployment'}
missing = expected - set(df.columns)
assert not missing, f"Missing columns: {missing}"

# Convenience
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month
df['Quarter'] = df['Date'].dt.quarter
df['Semester'] = np.where(df['Month']<=6, 'H1', 'H2')

# ----------------------------
# TASK 1: Which store has maximum sales (total)
# ----------------------------
store_total_sales = df.groupby('Store', as_index=False)['Weekly_Sales'].sum().rename(columns={'Weekly_Sales':'Total_Sales'})
store_with_max_sales = store_total_sales.sort_values('Total_Sales', ascending=False).head(1)

# ----------------------------
# TASK 2: Which store has maximum standard deviation & coefficient (std/mean)
# ----------------------------
store_stats = df.groupby('Store', as_index=False)['Weekly_Sales'].agg(['mean','std']).reset_index()
store_stats['coef_mean_to_std'] = store_stats['mean'] / store_stats['std']
store_with_max_std = store_stats.sort_values('std', ascending=False).head(1)

# ----------------------------
# TASK 3: Store(s) with good quarterly growth in Q3 2012
# Definition: (Q3 2012 total vs Q2 2012 total) % growth
# ----------------------------
df_2012 = df[df['Year']==2012].copy()
q_sales_2012 = df_2012.pivot_table(index='Store', columns='Quarter', values='Weekly_Sales', aggfunc='sum').fillna(0)
if 2 in q_sales_2012.columns and 3 in q_sales_2012.columns:
    q_sales_2012['Q3_growth_vs_Q2'] = (q_sales_2012[3] - q_sales_2012[2]) / q_sales_2012[2].replace(0, np.nan)
    good_growth = q_sales_2012.sort_values('Q3_growth_vs_Q2', ascending=False).dropna(subset=['Q3_growth_vs_Q2'])
else:
    good_growth = pd.DataFrame(columns=['Q3_growth_vs_Q2'])

# ----------------------------
# TASK 4: Holidays with higher sales than mean non-holiday sales (all stores)
# ----------------------------
mean_non_holiday = df.loc[df['Holiday_Flag']==0, 'Weekly_Sales'].mean()
# Aggregate each holiday week date across all stores; mark dates as "holiday dates"
holiday_weeks = df.loc[df['Holiday_Flag']==1].groupby('Date', as_index=False)['Weekly_Sales'].mean()
holidays_higher = holiday_weeks[holiday_weeks['Weekly_Sales'] > mean_non_holiday].sort_values('Weekly_Sales', ascending=False)

# ----------------------------
# TASK 5: Monthly & Semester views (units = Weekly_Sales; we'll sum)
# ----------------------------
monthly_view = df.groupby(['Year','Month'], as_index=False)['Weekly_Sales'].sum().rename(columns={'Weekly_Sales':'Monthly_Sales'})
semester_view = df.groupby(['Year','Semester'], as_index=False)['Weekly_Sales'].sum().rename(columns={'Weekly_Sales':'Semester_Sales'})

# ---- Print concise insights ----
print("\nStore with maximum total sales:")
print(store_with_max_sales.to_string(index=False))

print("\nStore with maximum sales standard deviation:")
print(store_with_max_std[['Store','mean','std','coef_mean_to_std']].to_string(index=False))

print("\nQ3 2012 growth vs Q2 2012 by store (top 10):")
print(good_growth[['Q3_growth_vs_Q2']].head(10).to_string())

print(f"\nMean non-holiday sales (all stores): {mean_non_holiday:,.2f}")
print("Holiday weeks (dates) with higher-than-mean non-holiday sales:")
print(holidays_higher.to_string(index=False))

print("\nMonthly sales (first 10 rows):")
print(monthly_view.head(10).to_string(index=False))

print("\nSemester (H1/H2) sales by year:")
print(semester_view.to_string(index=False))


Store with maximum total sales:
 Store  Total_Sales
    20 301397792.46

Store with maximum sales standard deviation:
 Store         mean           std  coef_mean_to_std
    14 2.020978e+06 317569.949476          6.363884

Q3 2012 growth vs Q2 2012 by store (top 10):
Quarter  Q3_growth_vs_Q2
Store                   
7               0.133308
16              0.084884
35              0.044666
26              0.039555
39              0.024784
41              0.024570
44              0.024346
24              0.016521
40              0.011428
23              0.008254

Mean non-holiday sales (all stores): 1,041,256.38
Holiday weeks (dates) with higher-than-mean non-holiday sales:
      Date  Weekly_Sales
2011-11-25  1.479858e+06
2010-11-26  1.462689e+06
2012-02-10  1.111320e+06
2010-02-12  1.074148e+06
2012-09-07  1.074001e+06
2011-02-11  1.051915e+06

Monthly sales (first 10 rows):
 Year  Month  Monthly_Sales
 2010      2   190332983.04
 2010      3   181919802.50
 2010      4   231412368.0

In [ ]:
from sklearn.model_selection import TimeSeriesSplit
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import mean_absolute_percentage_error, mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
import numpy as np

# ----------------------------
# FILTER STORE 1 & FEATURE ENGINEERING
# ----------------------------
s1 = df[df['Store']==1].copy().sort_values('Date')
s1 = s1.reset_index(drop=True)

# index starting at 1 for earliest date across ALL data (as required)
global_start = df['Date'].min()
s1['date_index'] = (s1['Date'] - global_start).dt.days // 7 + 1  # weekly index
s1['days_since_start'] = (s1['Date'] - global_start).dt.days

# (optional) simple seasonality features from weekly data
s1['weekofyear'] = s1['Date'].dt.isocalendar().week.astype(int)
s1['year'] = s1['Date'].dt.year

feat_cols = [
    'date_index',           # required
    'days_since_start',     # required "days"
    'Temperature',
    'Fuel_Price',
    'CPI',
    'Unemployment',
    'Holiday_Flag',
    'weekofyear'            # mild seasonal signal
]

s1 = s1.dropna(subset=feat_cols + ['Weekly_Sales']).copy()

X = s1[feat_cols].values
y = s1['Weekly_Sales'].values
sample_w = np.where(s1['Holiday_Flag']==1, 5.0, 1.0)  # holiday weighting

# ----------------------------
# TIME-AWARE TRAIN/TEST SPLIT
# (last 20% as test to respect chronology)
# ----------------------------
n = len(s1)
split_idx = int(n*0.8)
X_train, X_test = X[:split_idx], X[split_idx:]
y_train, y_test = y[:split_idx], y[split_idx:]
w_train = sample_w[:split_idx]
w_test  = sample_w[split_idx:]

# ----------------------------
# MODELS
# ----------------------------
candidates = {
    'LinearRegression': make_pipeline(StandardScaler(with_mean=True, with_std=True), LinearRegression()),
    'Ridge':            make_pipeline(StandardScaler(with_mean=True, with_std=True), Ridge(alpha=5.0, random_state=0)),
    'Lasso':            make_pipeline(StandardScaler(with_mean=True, with_std=True), Lasso(alpha=0.001, random_state=0, max_iter=20000))
}

results = []
for name, model in candidates.items():
    model.fit(X_train, y_train, **({'linearregression__sample_weight': w_train} if name=='LinearRegression' else
                                   {'ridge__sample_weight': w_train} if name=='Ridge' else
                                   {'lasso__sample_weight': w_train}))
    y_pred = model.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    mape = mean_absolute_percentage_error(y_test, y_pred)
    results.append((name, rmse, mape))

res_df = pd.DataFrame(results, columns=['Model','RMSE','MAPE']).sort_values('RMSE')
print("\nStore 1 — test set performance (lower is better):")
print(res_df.to_string(index=False))

best_name = res_df.iloc[0]['Model']
best_model = candidates[best_name]
print(f"\nSelected model: {best_name}")

# Fit on full data (if you want a final model for deployment)
best_model.fit(X, y, **({'linearregression__sample_weight': sample_w} if best_name=='LinearRegression' else
                         {'ridge__sample_weight': sample_w} if best_name=='Ridge' else
                         {'lasso__sample_weight': sample_w}))

# Example: next-week forecast (naive feature extension)
# Replace the below with your actual future covariates if available
last_row = s1.iloc[-1:].copy()
future = last_row.copy()
future['Date'] = last_row['Date'] + pd.Timedelta(days=7)
future['date_index'] = last_row['date_index'] + 1
future['days_since_start'] = (future['Date'] - global_start).dt.days
future['weekofyear'] = future['Date'].dt.isocalendar().week.astype(int)
# Keep other drivers equal to last observed unless you have forecasts
X_future = future[feat_cols].values
y_next_pred = best_model.predict(X_future)[0]
print(f"\nNext-week forecast (Store 1, naive drivers): {y_next_pred:,.2f}")


Store 1 — test set performance (lower is better):
           Model          RMSE     MAPE
           Ridge 160867.234128 0.089643
           Lasso 177619.610461 0.098974
LinearRegression 177619.626632 0.098974

Selected model: Ridge

Next-week forecast (Store 1, naive drivers): 1,619,580.96


In [ ]:
!ls

drive  sample_data
